In [1]:
import numpy as np
import pandas as pd
import json
import ijson

In [2]:
# Đường dẫn của data
file_name = r'D:\Learning\ML\BTL\get_raw_data\raw_data\total_match_timeline.json'
status_per_min_file_name = r'D:\Learning\ML\BTL\preprocessing\clean_data\status_per_min.json'
entries_win_rate_file_name = r'D:\Learning\ML\BTL\preprocessing\clean_data\entries_win_rate_at_next_match.json'
champion_win_rate_file_name = r'D:\Learning\ML\BTL\preprocessing\clean_data\champion_win_rate.json'
detail_match_file_name = r'D:\Learning\ML\BTL\get_raw_data\raw_data\detail_match_full.json'

In [3]:
with open(entries_win_rate_file_name, encoding='utf-8') as file:
    entries_win_rate = json.load(file)

In [4]:
with open(champion_win_rate_file_name, encoding='utf-8') as file:
    champions_win_rate = json.load(file)

In [5]:
with open(detail_match_file_name, encoding='utf-8') as file:
    detail_matches = json.load(file)

In [6]:
# Hàm flatten dict
def flatten_data(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

### 1. Chênh lệch Lượng vàng chênh lệch giữa xanh và đỏ mỗi phút

In [7]:
# Hàm tính lượng vàng chênh lệch giữa đội xanh và đỏ mỗi phút
def get_gold_difference_between_blue_and_red_team_per_minute(frames):
    gold_difference_df = {}
    # Duyệt qua từng phút từ 0 đến 40
    for minute in range(41):  # Bao gồm cả phút 40
        if minute < len(frames):  # Nếu frame tồn tại
            frame = frames[minute]
            total_gold_blue = 0
            total_gold_red = 0

            # Duyệt qua từng người chơi trong frame
            for participant_id, data in frame["participantFrames"].items():
                participant_id = int(participant_id)
                total_gold = data.get("totalGold", 0)  # Lấy giá trị, mặc định là 0 nếu không tồn tại
                
                if 1 <= participant_id <= 5:  # Đội xanh
                    total_gold_blue += total_gold
                elif 6 <= participant_id <= 10:  # Đội đỏ
                    total_gold_red += total_gold

            # Tính chênh lệch vàng
            gold_difference_df[f"gold_difference_at_{minute}"] = total_gold_blue - total_gold_red
        else:
            # Gán giá trị None nếu không có dữ liệu tại phút này
            gold_difference_df[f"gold_difference_at_{minute}"] = None

    return gold_difference_df


### 2. Trạng thái trụ mỗi đội, mỗi loại, mỗi phút

In [8]:
# Hàm trạng thái trụ mỗi đội, mỗi loại, mỗi phút
def calculate_turret_status(frames):
    initial_turrets = {
        "blue": {
            "MID_LANE": {"INNER_TURRET": 1, "OUTER_TURRET": 1, "BASE_TURRET": 1, "NEXUS_TURRET": 2},
            "TOP_LANE": {"INNER_TURRET": 1, "OUTER_TURRET": 1, "BASE_TURRET": 1},
            "BOT_LANE": {"INNER_TURRET": 1, "OUTER_TURRET": 1, "BASE_TURRET": 1},
        },
        "red": {
            "MID_LANE": {"INNER_TURRET": 1, "OUTER_TURRET": 1, "BASE_TURRET": 1, "NEXUS_TURRET": 2},
            "TOP_LANE": {"INNER_TURRET": 1, "OUTER_TURRET": 1, "BASE_TURRET": 1},
            "BOT_LANE": {"INNER_TURRET": 1, "OUTER_TURRET": 1, "BASE_TURRET": 1},
        }
    }

    # Kết quả lưu trạng thái trụ theo từng phút
    turret_status = {}


    # Duyệt qua từng frame (mỗi frame là 1 phút)
    for minute in range(41):  # Bao gồm cả phút 40
        if minute < len(frames):  # Nếu frame tồn tại
            frame = frames[minute]

            # Duyệt qua các lane và các loại trụ để sao chép trạng thái ban đầu cho phút đầu tiên
            for team in ["blue", "red"]:
                for lane, turrets in initial_turrets[team].items():
                    for tower_type, count in turrets.items():
                        # Cập nhật trạng thái trụ cho phút hiện tại
                        key = f"{team}_{lane}_{tower_type}_at_{minute}"
                        turret_status[key] = count

            # Lấy danh sách sự kiện trong frame
            events = frame.get("events", [])
            for event in events:
                # Kiểm tra nếu sự kiện là phá hủy trụ
                if event.get("type") == "BUILDING_KILL" and event.get("towerType") and event.get("laneType"):
                    team_id = event.get("teamId")
                    tower_type = event.get("towerType")
                    lane_type = event.get("laneType")

                    # Xác định đội (100 = blue, 200 = red)
                    team = "blue" if team_id == 100 else "red"

                    # Xác định key cho trụ bị phá hủy
                    key = f"{team}_{lane_type}_{tower_type}_at_{minute}"

                    # Giảm số lượng trụ trong lane và loại trụ tương ứng
                    if key in turret_status:
                        turret_status[key] -= 1
                        initial_turrets[team][lane_type][tower_type] -= 1
        else:
            for team in ["blue", "red"]:
                for lane, turrets in initial_turrets[team].items():
                    for tower_type, count in turrets.items():
                        # Cập nhật trạng thái trụ cho phút hiện tại
                        key = f"{team}_{lane}_{tower_type}_at_{minute}"
                        turret_status[key] = count
    return turret_status


### 3. Chênh lệch cấp độ mỗi đường, mỗi phút

In [9]:
# Hàm tính chênh lệch cấp độ mỗi đường giữa xanh và đỏ mỗi phút
def calculate_level_difference_by_lane(frames):
    # Lưu trữ chênh lệch cấp độ mỗi đường, mỗi phút
    level_diff = {}

    # Định nghĩa các participantId cho các đường
    lane_participants = {
        "TOP": [1, 6],
        "JUNGLE": [2, 7],
        "MID": [3, 8],
        "BOT": [4, 9],
        "SUPPORT": [5, 10]
    }

    # Duyệt qua từng phút từ 0 đến 40
    for minute in range(41):  # Bao gồm cả phút 40
        if minute < len(frames):  # Nếu frame tồn tại
            frame = frames[minute]
            blue_team_levels = []
            red_team_levels = []
            for participantFrameID, participantFrame in frame["participantFrames"].items():
                participant_id = participantFrame.get("participantId")
                level = participantFrame.get("level")
                # Xác định đội (100 = blue, 200 = red)
                if participant_id <= 5:  # Đội xanh
                    blue_team_levels.append(level)
                else:  # Đội đỏ
                    red_team_levels.append(level)

            # Lưu trữ cấp độ của các người chơi trong từng đường
            for lane, participants in lane_participants.items():
                key = f"level_difference_{lane}_at_{minute}"  # minute + 1 vì phút bắt đầu từ 0
                level_diff[key] = blue_team_levels[participants[0] - 1] - red_team_levels[participants[1] - 6]
        else:  # Nếu frame không tồn tại (vượt quá thời lượng trận đấu)
            for lane in lane_participants.keys():
                key = f"level_difference_{lane}_at_{minute}"
                level_diff[key] = None
    return level_diff


### 4. Trạng thái quái khủng, mỗi đội, mỗi phút

In [10]:
# Hàm tính chênh lệch quái khủng mỗi loại, mỗi phút giữa đội xanh và đỏ
def calculate_elite_monster_diff(frames):
    # Khởi tạo trạng thái chênh lệch quái khủng ban đầu
    initial_elite_monster_diff = {
        "HORDE": 0,
        "RIFTHERALD": 0,
        "FIRE_DRAGON": 0,
        "WATER_DRAGON": 0,
        "EARTH_DRAGON": 0,
        "AIR_DRAGON": 0,
        "CHEMTECH_DRAGON": 0,
        "HEXTECH_DRAGON": 0,
        "ELDER_DRAGON": 0,
        "BARON_NASHOR": 0
    }

    # Kết quả lưu trạng thái quái khủng theo từng phút
    elite_monster_status = {}

    # Duyệt qua từng phút từ 0 đến 40
    for minute in range(41):  # Bao gồm cả phút 40
        if minute < len(frames):  # Nếu frame tồn tại
            frame = frames[minute]

            # Lưu trạng thái quái khủng hiện tại cho phút này
            for monster_type in initial_elite_monster_diff.keys():
                key = f"{monster_type}_diff_at_{minute}"
                elite_monster_status[key] = initial_elite_monster_diff[monster_type]

            # Lấy danh sách sự kiện trong frame
            events = frame.get("events", [])

            for event in events:
                # Kiểm tra nếu sự kiện là ăn quái
                if event.get("type") == "ELITE_MONSTER_KILL":
                    team_id = event.get("killerTeamId")

                    # Xác định đội (100 = blue, 200 = red)
                    team = "blue" if team_id == 100 else "red"

                    monster_type = event.get("monsterType", "")
                    sub_monster_type = event.get("monsterSubType", "")

                    if monster_type == "DRAGON":
                        if sub_monster_type:
                            # Xử lý chênh lệch theo loại rồng
                            key = f"{sub_monster_type}_diff_at_{minute}"
                            if team == "blue":
                                initial_elite_monster_diff[sub_monster_type] += 1
                            else:
                                initial_elite_monster_diff[sub_monster_type] -= 1
                            elite_monster_status[key] = initial_elite_monster_diff[sub_monster_type]
                    else:
                        # Xử lý chênh lệch cho quái khủng không phải rồng
                        key = f"{monster_type}_diff_at_{minute}"
                        if team == "blue":
                            initial_elite_monster_diff[monster_type] += 1
                        else:
                            initial_elite_monster_diff[monster_type] -= 1
                        elite_monster_status[key] = initial_elite_monster_diff[monster_type]
        else:
            # Nếu không có frame tại phút này, gán trạng thái hiện tại
            for monster_type, count in initial_elite_monster_diff.items():
                key = f"{monster_type}_diff_at_{minute}"
                elite_monster_status[key] = count

    return elite_monster_status

### 5. Tỷ lệ thắng của người chơi mỗi đường, mỗi đội

In [11]:
def get_entry_win_rate_at_next_match(participants):
    entry_win_rate_at_next_match = {}
    for participant in participants:
        participantID = participant['participantId']
        puuid = participant['puuid']
        team = 'blue' if participantID <= 5 else 'red'
        if participantID == 1 or participantID == 6:
            lane = 'TOP'
        elif participantID == 2 or participantID == 7:
            lane = 'JUNGLE'
        elif participantID == 3 or participantID == 8:
            lane = 'MID'
        elif participantID == 4 or participantID == 9:
            lane = 'BOT'
        elif participantID == 5 or participantID == 10:
            lane = 'SUPPORT'
        key = f"{team}_{lane}_win_rate"
        entry_win_rate_at_next_match[key] = entries_win_rate[puuid] if puuid in entries_win_rate.keys() else 50.00
    return entry_win_rate_at_next_match

### 6. Tỷ lệ thắng của tướng mỗi đường mỗi đội

In [12]:
def get_champion_win_rate(detail_match):
    champion_win_rate = {}
    champion_picks = detail_match['team_100_picks'] + detail_match['team_200_picks']
    team = ''
    lane = ''
    for i in range(len(champion_picks)):
        championID = champion_picks[i]
        team = 'blue' if i <= 4 else 'red'
        if i == 0 or i == 5:
            lane = 'TOP'
        elif i == 1 or i == 6:
            lane = 'JUNGLE'
        elif i == 2 or i == 7:
            lane = 'MID'
        elif i == 3 or i == 8:
            lane = 'BOT'
        elif i == 4 or i == 9:
            lane = 'SUPPORT'
        key = f"{team}_{lane}_CHAMPION_win_rate"
        champion_win_rate[key] = champions_win_rate[str(championID)]['win_rate']
    return champion_win_rate

### 7. Lấy nhãn dữ liệu

In [13]:
def get_label(detail_match):
    return {'label' : detail_match['result']}

### Mở file và ghi dữ liệu

In [14]:
status_per_min = []

In [15]:
count = 0
with open(file_name, 'r', encoding='utf-8') as file:
    # Sử dụng ijson để tạo một iterator trên danh sách trong JSON
    objects = ijson.items(file, 'item')  # 'item' tương ứng với từng phần tử trong danh sách gốc
    
    for obj in objects:
        # Mỗi `obj` là một dict chứa 'frames' và 'participant'
        frames = obj.get('frames', None)
        participant = obj.get('participants', None)

        # Các phép tính và truy vấn thông tin cần thiết
        monster_status_per_min = calculate_elite_monster_diff(frames)
        gold_difference_per_min = get_gold_difference_between_blue_and_red_team_per_minute(frames)
        turret_status_per_min = calculate_turret_status(frames)
        level_status_per_min = calculate_level_difference_by_lane(frames)
        entry_win_rate_at_next_match = get_entry_win_rate_at_next_match(participant)
        champion_win_rate = get_champion_win_rate(detail_matches[count])
        label = get_label(detail_matches[count])
        count += 1

        # Cập nhật trạng thái và tính toán giá trị
        tmp_status_per_min = {}
        tmp_status_per_min.update(**entry_win_rate_at_next_match, **champion_win_rate, 
                                  **level_status_per_min, **turret_status_per_min, 
                                  **gold_difference_per_min, **monster_status_per_min, **label)
        
        status_per_min.append(tmp_status_per_min)

In [16]:
len(status_per_min)

21483

In [17]:
with open(status_per_min_file_name, 'w', encoding='utf-8') as file:
    json.dump(status_per_min, file, ensure_ascii=False, indent=4)